# Creation_data

- Script a vocation de récupérer des données de fichiers HTML
- Auteur : Benjamin ALLEAU / benjamin.fraime@gmail.com

# Packages

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

# Fonctions

In [6]:

# Charger le fichier HTML
def recuperateur_resultat(path_file):
    with open(path_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Extraire les matchs
    games = []
    match_elements = soup.find_all("div", class_="Match__Container-sc-exmma4-0")

    for match in match_elements:
        court = match.find("div", class_="subtitle").text.strip().replace("Court ", "")
        teams = match.find_all("div", class_="Match__TeamContainer-sc-exmma4-3")[0]
        # Utiliser une regex pour extraire les noms des joueurs
        equipes = re.findall(r'>([\wÀ-ÿ\.\-]+\s[\wÀ-ÿ\.\-]+/[\wÀ-ÿ\.\-]+\s[\wÀ-ÿ\.\-]+)<', str(teams))

        if len(equipes) == 2:
            eq1_joueurs = equipes[0].split("/")
            eq2_joueurs = equipes[1].split("/")

        ## Recuperation des scores
        scores = match.find_all("input", class_="MatchGame__StyledInput-sc-1asd8lj-1")
        scores1 = str(scores[0])
        match = re.search(r'value="(\d+)"', scores1)
        resultat1 = match.group(1) if match else None
        scores2 = str(scores[1])
        match = re.search(r'value="(\d+)"', scores2)
        resultat2 = match.group(1) if match else None  
        games.append([court,eq1_joueurs[0].strip(),eq1_joueurs[1].strip(),eq2_joueurs[0].strip(),eq2_joueurs[1].strip(),resultat1,resultat2])
            

    df = pd.DataFrame(games, columns=["Num Court", "Eq1 Joueur 1", "Eq1 Joueur 2", "Eq2 Joueur 1", "Eq2 Joueur 2", "Score Eq1", "Score Eq2"])
    return df

def apply_function_to_html_files(directory, function):
    """
    Applique une fonction à tous les fichiers HTML dans un répertoire donné.

    Args:
        directory (str): Le chemin du répertoire contenant les fichiers HTML.
        function (callable): La fonction à appliquer aux fichiers HTML.
    """
    # Parcourir tous les fichiers dans le répertoire
    list_res = []
    i = 1
    for filename in os.listdir(directory):
        print(i)
        print(filename)
        # Vérifier si le fichier a l'extension .html
        if filename.endswith('.html'):
            # Construire le chemin complet du fichier
            file_path = os.path.join(directory, filename)
            # Appliquer la fonction au fichier
            res_temp = function(file_path)
            res_temp["week"] = i
            print(len(res_temp))
            list_res.append(res_temp)
            i += 1

    return list_res


# Traitement des données 

In [7]:
directory_path = 'data'

# Appliquer la fonction example_function à tous les fichiers HTML dans le répertoire
list_res = apply_function_to_html_files(directory_path, recuperateur_resultat)
df_resultat = pd.concat(list_res)


1
Fwango1.html
30
2
Fwango1_files
2
Fwango2.html
24
3
Fwango2_files
3
Fwango3.html
30
4
Fwango3_files
4
Fwango4.html
30
5
Fwango4_files
5
Fwango5.html
30
6
Fwango5_files


# Affichage des résultats

In [8]:
df_resultat.head(5)

,Num Court,Eq1 Joueur 1,Eq1 Joueur 2,Eq2 Joueur 1,Eq2 Joueur 2,Score Eq1,Score Eq2,week
0,1,Théophane F.,Clement L.,Simon P.,Luis A.,17,18,1
1,2,Jean-Francois C.,Guillaume G.,Félix D.,Daphnée D.,14,18,1
2,3,Thomas L.,Raphael B.,Benjamin A.,Guillaume M.,17,18,1
3,4,Nil M.,Sophie L.,Maxime B.,Caroline N.,8,18,1
4,5,Vincent C.,Sarah J.,Solene M.,Fanette B.,10,18,1


In [9]:
df_resultat.to_pickle("data_preprocess/df_resultat.pkl")

In [10]:
df_resultat.to_csv("data_preprocess/df_resultat.csv",index=False)